# Create time-calibrated time series of land-cover probabilities

### Create time-calibrated land-cover probabilities using multi-year training data
* Many classes, for testing only woodland class
* subset of southern Chiquitania with training data fram that area
* Goal: extract a stack of annual land-cover probabilites

#### Long-term goal:
* smooth these probabilities using land-trendr

In [1]:
import ee  # the Earth Engine Python API library
ee.Initialize()
import ipyleaflet  # an interactive mapping "widget" for the notebook
from baumiTools import baumiEE, baumiVT
import json
import ogr, osr
import os
#import pandas as pd

## Load the shapefiles
1. The point shapefile, in which we have the training data
2. The landsat WRS-path/row data. These we need to do the sampling better.
3. Tiles, for which we want to download the classification probabilities

#### Also set:
* variables which we need for the identification in the shapefiles
* output-Folder and root-folder

In [ ]:
# Function to collect the spectra for the respective path row
def CollectSpectra(path, row):
    

In [8]:
root_folder = "D:/Chiquitania_CaseStudy/"
outFolder = root_folder
tileShp = baumiVT.CopyToMem(root_folder + "Processing_Frame_Tiles_testSub.shp")
landsat = baumiVT.CopyToMem(root_folder + "Landsat_WRS_testSub2.shp")
tilevar = "Id"
trainingData = baumiVT.CopyToMem(root_folder + "PointSample_FromRaster_points.shp")
classVar = "Class"

## Load the layers, create a coordinate transformation

In [12]:
# Get the 2 layers, build coordinate transformation
landsatLYR = landsat.GetLayer()
trainLYR = trainingData.GetLayer()
source_SR = landsatLYR.GetSpatialRef()
target_SR = trainLYR.GetSpatialRef()
trans = osr.CoordinateTransformation(source_SR, target_SR)
# Start loopin through the Landsat WRS-PathRows and get the spectral values for the points in there


#### Build a feature collection for the points that intersect with the Landsat footprints

In [108]:
landsatLYR.ResetReading()
# Create a feature collection from the path and row
landsatFEAT = landsatLYR.GetNextFeature()
landsatGEOM = landsatFEAT.GetGeometryRef()
landsatGEOM.Transform(trans)
# Set a spatial filter, check if there are any points in it, then loop over the points that remain from the filter
trainLYR.SetSpatialFilter(landsatGEOM)
if trainLYR.GetFeatureCount() == 0:
    landsatFEAT = landsatLYR.GetNextFeature()
else:
    yr_list = []
    p_list = []
    path = landsatFEAT.GetField('PATH')
    row = landsatFEAT.GetField('ROW')
    print(path, row)
    trainFEAT = trainLYR.GetNextFeature()
    while trainFEAT:
        trainGEOM = trainFEAT.GetGeometryRef()
        # Convert the CS to EPSG:4326
        source_SR = trainGEOM.GetSpatialReference()
        target_SR = osr.SpatialReference()
        target_SR.ImportFromEPSG(4326)
        trans = osr.CoordinateTransformation(source_SR, target_SR)
        trainGEOM.Transform(trans)
        # Get the attributes we want
        pid = trainFEAT.GetField('ID')
        cl = trainFEAT.GetField('Class')
        yr = trainFEAT.GetField('Year')
        yr_list.append(yr) # append to the year list, over which we map
        # Build the ee Feature
        geom_json = json.loads(trainGEOM.ExportToJson())
        geom_coord = geom_json['coordinates']
        geom_EE = ee.Geometry.Point(coords=geom_coord)
        if cl == 1:
            eeFeat = ee.Feature(geom_EE, {"ID": pid, "Class": 1, "Year": yr})
        else:
            eeFeat = ee.Feature(geom_EE, {"ID": pid, "Class": 0, "Year": yr})
        p_list.append(eeFeat)
        # Take the next feature
        trainFEAT = trainLYR.GetNextFeature()
    # Reset the reading for the trainLYR
    trainLYR.ResetReading()
    # Conver the lists of points to a ee.FeatureCollection()
    fc = ee.FeatureCollection(ee.List(p_list))
    # Balance the sample
    fc = balanceSample(fc, 1, 0)
    
fc.size().getInfo()

230 72


23

In [121]:
# Create a function that we map over to extract the training data
def Extract(year):
    # Filter the feature collection by year
    filt = ee.Filter(ee.Filter.eq('Year', ee.Number(year)))
    fc_filt = fc.filter(filt)
    
    seasMed = baumiEE.Calculate_Landsat_Seasonal_Median(year=year, startMonth=1, endMonth=3, path=path, row=row)
    for win in [[4,6], [7,9],[10,12]]:
        seasMed = seasMed.addBands(baumiEE.Calculate_Landsat_Seasonal_Median(year=year, startMonth=win[0], endMonth=win[1], path=path, row=row))
    # Rename the bands into sequential numbers
    seasMed = rename_bands(seasMed)
    #sam = seasMed.sampleRegions(collection=fc, properties=['Class'], scale=30, tileScale=1)
    def reduce_features(feats):
        return feats.set(seasMed.reduceRegion(reducer='mean', geometry=feats.geometry(), scale=30, tileScale=1))
    sam = fc_filt.map(reduce_features)
 
    
    return sam
    
    

In [112]:
f = ee.Filter(ee.Filter.eq('Year', 1985))
ft = fc.filter(f)
ft.size().getInfo(), fc.size().getInfo()

(7, 23)

In [123]:
ex85 = Extract(1985)
ex00 = Extract(2000)
ex15 = Extract(2015)


In [125]:
all = ex85.merge(ex00).merge(ex15)
all.getInfo()

{'type': 'FeatureCollection',
 'columns': {},
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-61.75421443653999, -17.97537241890794]},
   'id': '1_1_2_0',
   'properties': {'Class': 1,
    'ID': 18,
    'Year': 1985,
    'v001': 429.0,
    'v002': 554.0,
    'v003': 602.0,
    'v004': 2021.0,
    'v005': 1304.0,
    'v006': 626.0,
    'v007': 429.0,
    'v008': 554.0,
    'v009': 602.0,
    'v010': 2021.0,
    'v011': 1304.0,
    'v012': 626.0,
    'v013': 461.0,
    'v014': 622.0,
    'v015': 540.0,
    'v016': 2439.0,
    'v017': 1427.0,
    'v018': 681.0,
    'v019': 461.0,
    'v020': 622.0,
    'v021': 540.0,
    'v022': 2439.0,
    'v023': 1427.0,
    'v024': 681.0}},
  {'type': 'Feature',
   'geometry': {'type': 'Point',
    'coordinates': [-61.750736514517804, -17.91430337355846]},
   'id': '1_1_2_1',
   'properties': {'Class': 1,
    'ID': 147,
    'Year': 1985,
    'v013': 481.0,
    'v014': 621.0,
    'v015': 540.0,
    'v016': 2189.0,

In [129]:
bandNames = ee.Feature(all.first()).propertyNames().sort().remove('Class').remove('system:index').remove('ID').remove('Year')
print(bandNames.getInfo())

['v001', 'v002', 'v003', 'v004', 'v005', 'v006', 'v007', 'v008', 'v009', 'v010', 'v011', 'v012', 'v013', 'v014', 'v015', 'v016', 'v017', 'v018', 'v019', 'v020', 'v021', 'v022', 'v023', 'v024']


In [130]:
# Create our image for the year 2019
predIMG = baumiEE.Calculate_Landsat_Seasonal_Median(year=2019, startMonth=1, endMonth=3, path=path, row=row)
for win in [[4,6], [7,9],[10,12]]:
    predIMG = predIMG.addBands(baumiEE.Calculate_Landsat_Seasonal_Median(year=2019, startMonth=win[0], endMonth=win[1], path=path, row=row))
# Rename the bands into sequential numbers
predIMG = rename_bands(predIMG)


In [ ]:
yrs = ee.List(list(set(yr_list)))
al = yrs.map(Extract)

In [ ]:


#initFlag = 0
#while landsatFEAT:
# Take geometry, transform the CS into the Point-CS

# Set a spatial filter, check if there are any points in it, then loop over the points that remain from the filter
trainLYR.SetSpatialFilter(landsatGEOM)
if trainLYR.GetFeatureCount() == 0:
    landsatFEAT = landsatLYR.GetNextFeature()
else:
    #print("PathRow:", landsatFEAT.GetField('PR'), " found", str(trainLYR.GetFeatureCount()), "points")
    path = landsatFEAT.GetField('PATH')
    row = landsatFEAT.GetField('ROW')
# If there are points, then create a subList for the features
    list85s = []
    list00s = []
    list15s = []
    trainFEAT = trainLYR.GetNextFeature()
    while trainFEAT:
        trainGEOM = trainFEAT.GetGeometryRef()
        # Convert the CS to EPSG:4326
        source_SR = trainGEOM.GetSpatialReference()
        target_SR = osr.SpatialReference()
        target_SR.ImportFromEPSG(4326)
        trans = osr.CoordinateTransformation(source_SR, target_SR)
        trainGEOM.Transform(trans)
        # Get the attributes we want
        pid = trainFEAT.GetField('ID')
        cl = trainFEAT.GetField('Class')
        yr = trainFEAT.GetField('Year')
        # Build the ee Feature
        geom_json = json.loads(trainGEOM.ExportToJson())
        geom_coord = geom_json['coordinates']
        geom_EE = ee.Geometry.Point(coords=geom_coord)
        if cl == 1:
            eeFeat = ee.Feature(geom_EE, {"ID": pid, "Class": 1, "Year": yr})
        else:
            eeFeat = ee.Feature(geom_EE, {"ID": pid, "Class": 0, "Year": yr})
        if yr == 1985:
            list85s.append(eeFeat)
        if yr == 2000:
            list00s.append(eeFeat)
        if yr == 2015:
            list15s.append(eeFeat)
        # Take the next feature
        trainFEAT = trainLYR.GetNextFeature()
    # Reset the reading for the trainLYR
    trainLYR.ResetReading()
    # Conver the lists of points to a ee.FeatureCollection()
    fc85 = ee.FeatureCollection(ee.List(list85s))
    fc00 = ee.FeatureCollection(ee.List(list00s))
    fc15 = ee.FeatureCollection(ee.List(list15s))

    fc85 = balanceSample(fc85, 1, 0)
    fc00 = balanceSample(fc00, 1, 0)
    fc15 = balanceSample(fc15, 1, 0)
    # Now take the spectra --> make sure we rename the bands sequentially

    # Version I: through reduceRegion
    def red_fc_img(fc, year):
        t_median_L_85 = baumiEE.Calculate_Landsat_Seasonal_Median(year=year, startMonth=1, endMonth=3, path=path, row=row)
        for win in [[4, 6], [7, 9], [10, 12]]:
            t_median_L_85 = t_median_L_85.addBands(
                baumiEE.Calculate_Landsat_Seasonal_Median(year=year, startMonth=win[0], endMonth=win[1], path=path, row=row))
        def reduce_features(feats):
            return feats.set(t_median_L_85.reduceRegion(reducer='mean', geometry=feats.geometry(), scale=30, tileScale=1))
        sample = fc.map(reduce_features)
        return sample
    #trainingSample85 = red_fc_img(fc85, 1985)
    #trainingSample00 = red_fc_img(fc00, 2000)
    #trainingSample15 = red_fc_img(fc15, 2015)
    # Version II: through sampleREgion
    def samReg_fc_img(fc, year):
        seasMed = baumiEE.Calculate_Landsat_Seasonal_Median(year=year, startMonth=1, endMonth=3, path=path, row=row)
        for win in [[4,6], [7,9],[10,12]]:
            seasMed = seasMed.addBands(baumiEE.Calculate_Landsat_Seasonal_Median(year=year, startMonth=win[0], endMonth=win[1], path=path, row=row))
        # Rename the bands into sequential numbers
        seasMed = rename_bands(seasMed)
        sample = seasMed.sampleRegions(collection=fc, properties=['Class'], scale=30, tileScale=2)
        return sample
    trainingSample85 = samReg_fc_img(fc85, 1985)
    #trainingSample00 = samReg_fc_img(fc00, 2000)
    #trainingSample15 = samReg_fc_img(fc15, 2015)
#        # Attach to output lists
#        if initFlag == 0:
#            eeFeats85 = trainingSample85
#            eeFeats00 = trainingSample00
#            eeFeats15 = trainingSample15
#            initFlag = 1
#        else:
#            eeFeats85 = eeFeats85.merge(trainingSample85)
#            eeFeats00 = eeFeats00.merge(trainingSample00)
#            eeFeats15 = eeFeats15.merge(trainingSample15)
            # Get the next feature
            #tileFEAT = landsatLYR.GetNextFeature()
    # Merge the three together, write them to disc as geoJSON
#    allFeats = eeFeats85.merge(eeFeats00).merge(eeFeats15)
    # Export the training data as geojson
#    with open('D:/Chiquitania_CaseStudy/featureCollection2.geojson', 'wb') as f:
#        pickle.dump(allFeats, f)
#    f.close()

trainingSample85.getInfo()


In [8]:
val = trainingSample85.getInfo()
print(val['properties'])

{'band_order': ['v001', 'v002', 'v003', 'v004', 'v005', 'v006', 'v007', 'v008', 'v009', 'v010', 'v011', 'v012', 'v013', 'v014', 'v015', 'v016', 'v017', 'v018', 'v019', 'v020', 'v021', 'v022', 'v023', 'v024']}


In [ ]:
bandNames = median_L8.bandNames()
classifier = ee.Classifier.randomForest(100,0).setOutputMode('PROBABILITY').train(trainingSample,'Class',bandNames)
classification = median_L8.classify(classifier)#.multiply(100)
#bandNames.getInfo()

In [ ]:
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

Map = ipyleaflet.Map(center=(-14.438411, -63.743799), zoom=4,
                      layout={'height':'600px'},
                      scroll_wheel_zoom=True)
classific = ipyleaflet.TileLayer(name="classification", url=GetTileLayerUrl(
    classification.visualize(min=0, max=1, gamma=1.5)))
Landsat8 = ipyleaflet.TileLayer(name="Landsat8", url=GetTileLayerUrl(
    median_L8.visualize(min=0, max=8000, gamma=1.5,bands= ['NIR_2019-01-03', 'SWIR1_2019-01-03', 'R_2019-01-03'])))

Map.add_layer(Landsat8)
Map.add_layer(classific)

Map.add_control(ipyleaflet.LayersControl(position='topright'))

Map

## Helper functions

In [4]:
# Make an even sample between class 1 and 0
def balanceSample(fc, cl1, cl2):
    def SetClass(feat):
        return feat.set("Class", 1)
    def SetOther(feat):
        return feat.set("Class", 0)
    primeClass = fc.filter(ee.Filter.eq("Class", cl1))
    otherClass = fc.filter(ee.Filter.eq("Class", cl2))
    cl = primeClass.map(SetClass)
    oth = otherClass.map(SetOther)
    size_cl = cl.size().getInfo()
    size_oth = oth.size().getInfo()
    if size_cl > size_oth:
        ratio = size_oth / (size_cl + size_oth)
        cl = cl.randomColumn("random")
        cl = cl.filter(ee.Filter.lt("random", ratio))
    elif size_oth > size_cl:
        ratio = size_cl / (size_cl + size_oth)
        oth = oth.randomColumn("random")
        oth = oth.filter(ee.Filter.lt("random", ratio))
    else:
        cl = cl
        oth = oth
    # merge
    referenceData = oth.merge(cl)
    return referenceData

In [6]:
# Rename bands in a sequential way
def rename_bands(multiband):
    old_bandnames = multiband.bandNames()
    bandseq = ee.List.sequence(1, old_bandnames.size())
    def create_bandnames(i):
        return ee.String('v').cat(ee.Number(i).format('%03d'))
    new_bandnames = bandseq.map(create_bandnames)
    return multiband.select(old_bandnames, new_bandnames)